In [43]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import time
import datetime

## Подход 1: Градиентный бустинг

#### Считайте таблицу с признаками из файла features.csv. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [225]:
features = pd.read_csv('./features.csv', index_col='match_id')
features

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,4,3,0,-24.0,2032,0,1792,1975,48,63
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,3,2,0,-17.0,1734,1,2038,6,63,3
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,1,3,1,-15.0,2906,0,1796,1846,51,63


In [226]:
y = features['radiant_win']
drop_columns = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
              'barracks_status_radiant', 'barracks_status_dire', 'start_time']
features.drop(drop_columns, axis=1, inplace=True)

#### Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [94]:
for feature in features.columns:
    if features[feature].count() < features.count()[0]:
        print(feature, features[feature].count())

first_blood_time 77677
first_blood_team 77677
first_blood_player1 77677
first_blood_player2 53243
radiant_bottle_time 81539
radiant_courier_time 96538
radiant_flying_courier_time 69751
radiant_first_ward_time 95394
dire_bottle_time 81087
dire_courier_time 96554
dire_flying_courier_time 71132
dire_first_ward_time 95404


In [227]:
# Замените пропуски на нули с помощью функции fillna()
features.fillna(0, inplace=True)

In [ ]:
# Какой столбец содержит целевую переменную? Запишите его название.
radiant_win

##### Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30).

In [37]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
clf = GradientBoostingClassifier(n_estimators=10, random_state=42)
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.6648506879750012
Time elapsed: 0:00:36.086527


In [38]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
clf = GradientBoostingClassifier(n_estimators=20, random_state=42)
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.6824618768044435
Time elapsed: 0:01:13.150375


In [39]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
clf = GradientBoostingClassifier(n_estimators=30, random_state=42)
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.6900064710388155
Time elapsed: 0:01:46.211991


In [41]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
clf = GradientBoostingClassifier(n_estimators=40, random_state=42)
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.6939415596721178
Time elapsed: 0:02:25.135958


### Подход 2: логистическая регрессия


In [161]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [145]:
clf = LogisticRegression(penalty='l2') 
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.7164069956808498
Time elapsed: 0:00:02.848383


In [52]:
clf = LogisticRegression(penalty='l2', C=0.01) 
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.7164355801469666
Time elapsed: 0:00:02.417535


In [228]:
category = features[['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
               'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
features.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
               'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1, inplace=True)

In [229]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [230]:
clf = LogisticRegression(penalty='l2', C=0.01) 
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, features, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.7164426665509407
Time elapsed: 0:00:02.288418


In [152]:
# Выясните из данных, сколько различных идентификаторов героев существует в данной игре
print(len(category['r1_hero'].unique()))

108


In [231]:
# Воспользуемся подходом "мешок слов" для кодирования информации о героях.
X_pick = np.zeros((category.shape[0], 112))

for i, match_id in enumerate(category.index):
    for p in range(5):
        X_pick[i, category[f'r{(p+1)}_hero'][match_id] - 1] = 1
        X_pick[i, category[f'd{(p+1)}_hero'][match_id] - 1] = -1

In [232]:
pick = pd.DataFrame(X_pick)
features = pd.DataFrame(features)
data = pd.concat([features, pick], axis = 1)

In [233]:
data

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,-0.332256,-0.625222,-0.255162,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,0.578881,0.732454,-0.250795,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,-0.332256,0.224676,-0.726779,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,-1.243393,-1.170813,-1.242065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,-1.243393,-1.008757,-1.213680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,0.501314,0.834109,0.107949,0.637942,-0.537757,1.017574,-0.108632,-1.243393,-1.032165,-1.250798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97226,0.501314,0.987660,0.577666,0.637942,-0.537757,1.017574,-1.309804,-0.332256,-0.765671,-0.672194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97227,0.501314,0.292267,-1.309820,-1.131661,-0.537757,-0.578083,-1.309804,0.578881,0.465961,0.574534,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97228,-0.398181,-0.173682,-0.822866,-0.578660,-0.537757,2.613231,-0.909413,1.490017,1.377080,0.762308,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [234]:
scaler = StandardScaler()
df = scaler.fit_transform(data)

In [235]:
clf = LogisticRegression(penalty='l2', C=0.01) 
start_time = datetime.datetime.now()
print(np.mean(cross_val_score(clf, df, y, scoring='roc_auc', cv=cv)))
print('Time elapsed:', datetime.datetime.now() - start_time)

0.7518095215178435
Time elapsed: 0:00:05.124274


In [236]:
clf.fit(df, y)

LogisticRegression(C=0.01)

In [237]:
test = pd.read_csv('features_test.csv', index_col='match_id')
test.drop('start_time', axis=1, inplace=True)
test.fillna(0, inplace=True)
category = test[['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
               'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
               'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1, inplace=True)

In [238]:
X_pick = np.zeros((category.shape[0], 112))

for i, match_id in enumerate(category.index):
    for p in range(5):
        X_pick[i, category[f'r{(p+1)}_hero'][match_id] - 1] = 1
        X_pick[i, category[f'd{(p+1)}_hero'][match_id] - 1] = -1

In [239]:
test.reset_index(drop=True, inplace=True)
test = scaler.fit_transform(test)
pick = pd.DataFrame(X_pick)
test = pd.DataFrame(test)
data = pd.concat([test, pick], axis = 1)

In [240]:
data

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,-0.304443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,0.635111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,-1.179583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,-0.442155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
4,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,-0.302222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,1.394934,1.448970,1.699166,1.733795,-0.532732,1.003884,-0.093180,0.567766,0.505291,-0.364415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17173,-0.445442,-0.867378,-1.180227,-1.145090,-0.532732,-0.593200,1.500200,-0.368985,-0.091940,0.632890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
17174,-1.365630,-1.174215,-1.408682,-1.255816,-0.532732,-0.593200,-0.093180,-0.368985,-0.875579,-0.919706,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
17175,0.474746,1.076510,0.050412,-0.370005,0.968865,2.600967,-0.093180,-0.368985,-0.303685,-0.575426,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [241]:
scaler = StandardScaler()
df = scaler.fit_transform(data)

In [242]:
pred = clf.predict_proba(data)[:, 1]

In [243]:
pred.min()

0.006563751923887036

In [244]:
pred.max()

0.9955659138934668